# Question 1

(30 points)

Tracking Error is a key component in evaluating the performance of an exchange traded fund. A lower tracking error is one of the key selling points for any ETF.

An estimate of daily tracking error (TEd)can be based on the following formula:

![STRAT-06.ipynb.png](STRAT-06.ipynb.png)

where R<sub>b,t</sub> denotes the daily return of the benchmark at time t, R<sub>x,t</sub> denotes the daily return of the fund x at time t and N denotes the total number of trading days considered. TE<sub>d</sub> is usually scaled by a factor of √252 to obtain an estimate of annualized tracking error(TE).

The following files have been provided along with this assignment :

* NIFTY-TotalReturnsIndex.csv : Nifty Total Returns Index (2016 and 2017)
* Reliance Nifty ETF.xlsx : Historical NAV data for Reliance Nifty ETF (2016 and 2017) 
* Kotak Nifty ETF.xlsx : Historical NAV data for Kotak Nifty ETF(2016 and 2017)
* HDFC Nifty ETF.xlsx : Historical NAV data for HDFC Nifty ETF(2016 and 2017)
* UTI Nifty ETF.xlsx : Historical NAV data for UTI Nifty ETF(2016 and 2017)

Compute the annualized tracking error for Reliance, Kotak, HDFC and UTI ETFs in 2016 and 2017 separately.

Deliverable

* R code or Python code used for the analysis.
* Arrange the four funds (Reliance, Kotak, HDFC and UTI) in the increasing order of TE in 2016 and
2017.
* Out of the four funds, which ones have shown an increase in Annualized TE from 2016 to 2017 ?
* Out of the four funds, which ones have shown an decrease in Annualized TE from 2016 to 2017 ?
* Put in comments towards the end of your code that shows the results of the above questions

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

nifty = pd.read_csv("Q1-Data/NIFTY-TotalReturnsIndex.csv" , parse_dates=True , index_col=0)

hdfc = pd.read_excel("Q1-Data/HDFC Nifty ETF.xlsx" ,  sheet_name=0,skiprows=4 , index_col=0)
kotak = pd.read_excel("Q1-Data/Kotak Nifty ETF.xlsx" ,  sheet_name=0,skiprows=3 , index_col=0)
reliance = pd.read_excel("Q1-Data/Reliance Nifty ETF.xlsx" ,  sheet_name=0,skiprows=2 , index_col=0)
uti = pd.read_excel("Q1-Data/UTI Nifty ETF.xlsx" ,  sheet_name=0,skiprows=2 , index_col=0)

all_stocks = pd.concat([nifty,hdfc,kotak,reliance,uti],axis=1 )
all_stocks.columns = ['nifty','hdfc','kotak','reliance','uti']
daily_returns = all_stocks.pct_change()

def tracking_error(s1,s2):
    rows = s1.shape[0]
    numerator = np.sum(np.square(s1-s2))
    te = np.sqrt(252*numerator/(rows-1))
    return te

def yearly_tes(df , year,benchmark='nifty',):
    te = {}
    df_year = df.loc[year]
    for col in df.columns:
        if col != benchmark:
            te[col] = tracking_error(df_year[benchmark] , df_year[col])
    return te

print('Tracking errors for 2016:' , sorted(yearly_tes(daily_returns,'2016','nifty').items(),key=lambda x:x[1]))
print('Tracking errors for 2017:' , sorted(yearly_tes(daily_returns,'2017','nifty').items(),key=lambda x:x[1]))

Tracking errors for 2016: [('reliance', 0.03913706701044409), ('kotak', 0.05172558145446783), ('uti', 0.1059947871088013), ('hdfc', 0.11565250915328808)]
Tracking errors for 2017: [('reliance', 0.022432398648723574), ('kotak', 0.03874427807900707), ('hdfc', 0.05968149278754654), ('uti', 0.08399025321485849)]


In [5]:
daily_returns['uti_ret'] = ((all_stocks['uti'] - all_stocks['uti'].shift(1)) / all_stocks['uti'].shift(1))
daily_returns.tail()

,nifty,hdfc,kotak,reliance,uti,uti_ret
Date,,,,,,
2017-12-22,0.005050,0.002774,0.001985,0.004532,0.000102,0.000102
2017-12-26,0.003667,-0.000617,0.002359,0.003196,0.006527,0.006527
2017-12-27,-0.003867,0.006144,-0.001130,-0.000406,-0.000202,-0.000202
2017-12-28,-0.001226,-0.004805,-0.004240,-0.002301,0.000975,0.000975
2017-12-29,0.005036,0.000243,0.005773,0.004260,0.002619,0.002619


In [2]:
nifty.tail()

,Total Returns Index
Date,
2017-12-22,14330.49
2017-12-26,14383.04
2017-12-27,14327.42
2017-12-28,14309.85
2017-12-29,14381.92


In [3]:
daily_returns.head()

,nifty,hdfc,kotak,reliance,uti
Date,,,,,
2016-01-01,NaN,NaN,NaN,NaN,NaN
2016-01-04,-0.021585,-0.009551,-0.015030,-0.018322,-0.010616
2016-01-05,-0.000851,-0.007727,-0.005536,-0.000633,-0.001107
2016-01-06,-0.005608,0.008302,-0.002283,-0.005887,-0.006945
2016-01-07,-0.022312,-0.028263,-0.017756,-0.020684,-0.020146


In [4]:
uti.head()

,NAV
Date,
2016-01-01,794.05
2016-01-04,785.62
2016-01-05,784.75
2016-01-06,779.30
2016-01-07,763.60


# Question 2

(40 points)

This question deals with the computation of returns and sharpe ratio for a simple asset allocation strategy. The following are the details of the strategy

* Initial Capital : 100 Million INR
* Portfolio Allocation Start Date : January 1, 2016
* Portfolio Redemption Date : December 30, 2017
* Capital invested among Nifty BeES, Junior BeES and Gold BeES in the ratio of 5:2:3 on the Portfolio Allocation Start Date. The portfolio is rebalanced ONLY at the end of every quarter to bring the weights back to the initial allocation, i.e. 5:2:3. Note that the rebalancing should be done on the last day of each quarter and hence the close prices on the last working day of each quarter should be used for rebalancing portfolio.

Compute the sharpe ratio of your asset allocation strategy for 2016 and 2017. Assume zero transaction costs for quarterly rebalancing. Assume you can rebalance based on end of the day NAV.

The Historical NAV files for Nifty BeES, Junior BeES and Gold BeES are provided with the assignment. The following are the details :
    
* Nifty ETF.xlsx : Historical NAV data for Nifty ETF
* Junior ETF.xlsx : Historical NAV data for Junior ETF 
* Gold ETF.xlsx : Historical NAV data for Gold ETF

Deliverable

* R code or Python code used for the analysis.
* Annualized returns for the strategy in 2016 and 2017
* Sharpe Ratio for the strategy in 2016 and 2017
* Put in comments towards the end of your code that shows the results of the above questions

In [15]:
nifty_bees =  pd.read_excel("STRATS6-Q2-Data/Nifty ETF.xlsx" , sheet=0,skiprows =2 ,index_col=0)
junior_bees =  pd.read_excel("STRATS6-Q2-Data/Junior ETF.xlsx" , sheet=0,skiprows =3 ,index_col=0)
gold_bees =  pd.read_excel("STRATS6-Q2-Data/Gold ETF.xlsx" , sheet=0,skiprows =4 ,index_col=0)

q2_stocks = pd.concat([nifty_bees , junior_bees , gold_bees] , axis =1)

q2_stocks.columns = ['Nifty' ,'Junior','Gold']

init_capital = 1e8

nifty_shares = np.floor(0.5*init_capital/q2_stocks.loc['2016-01-01','Nifty'])
junior_shares = np.floor(0.2*init_capital/q2_stocks.loc['2016-01-01','Junior'])
gold_shares = np.floor(0.3*init_capital/q2_stocks.loc['2016-01-01','Gold'])

print(nifty_shares , junior_shares,gold_shares)

def get_quarter_end_prices(df , year , quarter ):
    prices = df[(df.index.quarter == quarter ) & (df.index.year == year)].iloc[-1,]
    return dict(prices)

def calc_curr_value(curr_prices , shares ):
    return np.round(np.sum([curr_prices[x]*shares[x] for x in shares.keys()]),2)

def get_share_counts(total_val , prices , proportions):
    shares = {stock:np.floor((total_val*proportions[stock])/prices[stock]) for stock in prices.keys() }
    return shares

curr_shares={2016:{1:{'Nifty':nifty_shares,'Junior':junior_shares,'Gold':gold_shares}}}

proportions = {'Nifty':0.5 , 'Junior' : 0.2, 'Gold':0.3}

yr_q_prices = {} 
yr_q_portfolio_vals = {}
for year in [2016,2017]:
    #print(curr_shares)
    for quarter in [1,2,3,4]:
        if yr_q_prices.get(year):
            yr_q_prices[year][quarter] = get_quarter_end_prices(q2_stocks , year,quarter)
            yr_q_portfolio_vals[year][quarter] = calc_curr_value(yr_q_prices[year][quarter] , curr_shares[year][quarter])
 
        else :
            yr_q_prices[year] = {quarter:get_quarter_end_prices(q2_stocks , year,quarter)}
            yr_q_portfolio_vals[year] = {quarter:calc_curr_value(yr_q_prices[year][quarter] , curr_shares[year][quarter])}
            
        
        if quarter != 4:
            curr_shares[year][quarter+1] = get_share_counts(yr_q_portfolio_vals[year][quarter],
                                                          yr_q_prices[year][quarter],
                                                           proportions)
        else:
            curr_shares[year+1] = {1 : get_share_counts(yr_q_portfolio_vals[year][quarter],
                                                          yr_q_prices[year][quarter],
                                                           proportions
                                                          ) }

62107.0 98222.0 13156.0


In [ ]:
df['return'] = np.log(q2_stocks['portfolio_val'] / q2_stocks['portfolio_val'].shift(1))

In [16]:
q2_returns = q2_stocks.pct_change()

vals = []

for year in [2016,2017 ]:
    for quarter in [1,2,3,4]:
        shares = curr_shares[year][quarter]
        counts = [shares['Nifty'] , shares['Junior'] , shares['Gold']]
        val = np.sum(q2_stocks.iloc[(q2_stocks.index.quarter==quarter) & (q2_stocks.index.year==year)] * counts ,axis=1)
        vals.append(val)

q2_stocks['portfolio_val'] = pd.concat(vals)

In [17]:
q2_stocks['portfolio_val'].pct_change().sum()

0.29181628126703407

In [18]:
total_returns = q2_stocks['portfolio_val'].pct_change().sum()
annualized_returns = 100*np.round(((1+total_returns)**(1/2))-1,5)
print(" Annualized returns of the portfolio: ", annualized_returns,'%')

 Annualized returns of the portfolio:  13.658000000000001 %


In [19]:
# Assuming a 6.5% annual risk free rate , daily risk free rate is 
riskfreerate = ((1+0.065)**(1/365))-1

#print(riskfreerate)
pf_returns = q2_stocks['portfolio_val'].pct_change()
sharpe_ratio = (pf_returns-riskfreerate).mean()/np.std((pf_returns-riskfreerate) , ddof=1)

print('sharpe_ratio  :', sharpe_ratio)

sharpe_ratio  : 0.07770729094992039


Annualized returns of the portfolio : 13.658% Sharpe ratio assuming a 6.5% annual risk free rate : 0.078

# Question 3

(30 points)

Assume you have an initial capital of 100 Million INR and the start date of your investment in January 1, 2016. Select any 10 stocks trading on National Stock Exchange as of January 1, 2016 and allocate equally amongst all the 10 stocks( 1/10th of your initial capital in each stock). Post the allocation on the start date, there is no rebalancing done until the end 2017. You redeem your portfolio at the end of 2017

Compute the daily portfolio returns from the start date till the redemption date. Do a linear regression of your portfolio returns on Nifty ETF daily returns and Junior ETF daily returns.

R<sub>P,t</sub> = β<sub>0</sub> + β<sub>1</sub> · R<sub>N,t</sub> + β<sub>2</sub> · R<sub>J,t</sub> + ε<sub>t</sub>

where R<sub>P,t</sub> denotes the daily return of your portfolio at time t, R<sub>N,t</sub> denotes the daily return of Nifty ETF at
time t and R<sub>J,t</sub> denotes the daily return of Junior ETF at time t.

The Historical NAV files for Nifty BeES, Junior BeES are provided with the assignment. 

* Nifty ETF.xlsx : Historical NAV data for Nifty ETF
* Junior ETF.xlsx : Historical NAV data for Junior ETF

Deliverable

* R code or Python code used for the analysis.
* Compute the coefficients β<sub>1</sub> and β<sub>2</sub>
* Compute of Portfolio return variation that is not explained by Nifty ETF and Junior ETF return
variation
* Put in comments towards the end of your code that lists out β<sub>1</sub> and β<sub>2</sub> and unexplained portfolio return
variation.

In [3]:
import numpy as np, pandas as pd,os,re,sys,nsepy as npy,datetime,statsmodels.api as sm
from os.path import join

#### File location and name: change them here
readpthstr = os.getcwd()+'//STRATS6-Q3-Data'
filenames = os.listdir(readpthstr)
absfilepath=[join(readpthstr, f) for f in filenames]
filepathlis = [f for f in absfilepath if os.path.isfile(f)]
ticker=[re.split('\W+',os.path.basename(f))[0] for f in filepathlis]
tickerdict=dict.fromkeys(ticker)
filecount=len(filepathlis)

colnam = ['Date', 'NAV']
coldtype = {'NAV': np.float32}
for i,tickr in enumerate(tickerdict):
    try:
        df=pd.read_excel(filepathlis[i], names=colnam, na_values=['Close','NAV','Date'])
        df=df.astype(dtype=coldtype).dropna().drop_duplicates('Date')
        df['ln_ret']=np.log(df['NAV']/df['NAV'].shift(1))
        df.set_index('Date',inplace=True)
        tickerdict[tickr]=df['ln_ret'].dropna()
    except:
        print("Unexpected error:", sys.exc_info()[1])
bmdf=pd.concat([x for x in tickerdict.values()],join='inner', axis=1,keys=['ln_ret_'+x for x in tickerdict.keys()])

#### Initial Setup
nsetick=['ASIANPAINT',
 'BAJAJ-AUTO',
 'BAJAJFINSV',
 'EICHERMOT',
 'HDFCBANK',
 'HEROMOTOCO',
 'HINDUNILVR',
 'ICICIBANK',
 'MOTHERSUMI',
 'TATAMOTORS']
initratio=[1,1,1,1,1,1,1,1,1,1]
investamt=100
initalloc = initratio/np.sum(initratio)
initalloc=dict(zip(nsetick,initalloc))

#### load ticker data using nsepy
nsedict=dict.fromkeys(nsetick)
for tick in nsedict:
    df=pd.DataFrame(npy.get_history(symbol=tick,
                             start=datetime.date(2016,1,1),
                            end=datetime.date(2017,12,31))['Close'])
    if not df.empty:
        df['ln_ret']=np.log(df['Close']/df['Close'].shift(1))*initalloc[tick]
        nsedict[tick]=df['ln_ret'].dropna()
    else:
        print("Query for ticker: {0} returned an empty dataframe!".format(tick))

#### final calculations
nsedf=pd.concat([x for x in nsedict.values()],join='inner', axis=1,keys=['w_lnret_'+x for x in nsedict.keys()])
portdf=nsedf.sum(axis=1).to_frame('Port_ln_ret')
ddf=portdf.merge(bmdf,how='inner',left_index=True,right_index=True).dropna()
X=sm.add_constant(ddf[['ln_ret_Junior','ln_ret_Nifty']])
Y=ddf['Port_ln_ret']
model=sm.OLS(Y,X)
results = model.fit()
print(results.params)
print('Unexplained Variation: ', 1-results.rsquared)


const            0.000046
ln_ret_Junior    0.244074
ln_ret_Nifty     0.887701
dtype: float64
Unexplained Variation:  0.2511915635134948


In [18]:
Y.tail()

Date
2017-12-22    0.004527
2017-12-26    0.001941
2017-12-27   -0.007647
2017-12-28   -0.003218
2017-12-29    0.009958
Name: Port_ln_ret, dtype: float64

In [7]:
nsedf.tail()

,w_lnret_ASIANPAINT,w_lnret_BAJAJ-AUTO,w_lnret_BAJAJFINSV,w_lnret_EICHERMOT,w_lnret_HDFCBANK,w_lnret_HEROMOTOCO,w_lnret_HINDUNILVR,w_lnret_ICICIBANK,w_lnret_MOTHERSUMI,w_lnret_TATAMOTORS
Date,,,,,,,,,,
2017-12-22,0.000009,0.001194,0.001388,0.000007,0.000446,-0.000346,0.000595,0.000364,0.000585,0.000285
2017-12-26,0.001114,0.001196,-0.000454,0.000001,-0.000443,0.000552,-0.000621,0.000488,-0.000412,0.000520
2017-12-27,-0.000148,-0.000893,-0.001440,-0.000200,-0.000623,-0.000714,0.000322,-0.001696,-0.001842,-0.000413
2017-12-28,-0.000406,-0.001323,-0.000387,-0.001194,0.001141,-0.001866,0.000151,0.000796,0.000784,-0.000916
2017-12-29,0.001225,0.001160,-0.001430,0.001690,-0.000301,0.001826,0.000981,-0.000413,0.002105,0.003116


In [4]:
df.tail()

,Close,ln_ret
Date,,
2017-12-22,422.00,0.000285
2017-12-26,424.20,0.000520
2017-12-27,422.45,-0.000413
2017-12-28,418.60,-0.000916
2017-12-29,431.85,0.003116


In [5]:
df['ln_ret']

Date
2016-01-01         NaN
2016-01-04   -0.006383
2016-01-05   -0.000692
2016-01-06   -0.002310
2016-01-07   -0.006303
2016-01-08    0.002813
2016-01-11    0.001920
2016-01-12   -0.001454
2016-01-13    0.002668
2016-01-14   -0.002866
2016-01-15   -0.003738
2016-01-18   -0.000323
2016-01-19    0.004019
2016-01-20   -0.003418
2016-01-21   -0.004085
2016-01-22    0.003278
2016-01-25   -0.002053
2016-01-27    0.001329
2016-01-28   -0.000849
2016-01-29    0.000805
2016-02-01    0.000474
2016-02-02   -0.002680
2016-02-03   -0.002847
2016-02-04    0.001610
2016-02-05    0.003502
2016-02-08   -0.004118
2016-02-09   -0.004053
2016-02-10   -0.006346
2016-02-11   -0.005354
2016-02-12    0.007660
                ...   
2017-11-17    0.001999
2017-11-20    0.000284
2017-11-21    0.000295
2017-11-22    0.001020
2017-11-23   -0.000550
2017-11-24   -0.000176
2017-11-27   -0.000968
2017-11-28   -0.001302
2017-11-29   -0.000482
2017-11-30   -0.002396
2017-12-01   -0.001308
2017-12-04    0.001184
2017-1

In [6]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            Port_ln_ret   R-squared:                       0.749
Model:                            OLS   Adj. R-squared:                  0.748
Method:                 Least Squares   F-statistic:                     731.8
Date:                Fri, 12 Apr 2019   Prob (F-statistic):          5.03e-148
Time:                        15:27:34   Log-Likelihood:                 1910.3
No. Observations:                 494   AIC:                            -3815.
Df Residuals:                     491   BIC:                            -3802.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          4.587e-05      0.000      0.200